# In this notebook, a unit square mesh is used to show some basic commands which we will be using in fenics implementation.  

* Different methods to import the mesh
* Print the cordinates, vertex, cells
* Save dof in xdmf file 
* Visualise the cell data and point data in Paraview

## Import dolfin module

In [1]:
from dolfin import *
import numpy as np

## Next two ways of using the mesh is shown
1. Importing mesh saved as xdmf file
2. Using the inbuilt mesh in dolfin

### Inbuilt mesh

In FEniCS we have following built-in meshes -
1. UnitIntervalMesh
2. UnitSquareMesh
3. RectangleMesh
4. UnitCubeMesh
5. BoxMesh

- Unit Interval Mesh - `mesh = UnitIntervalMesh(N)`  
N defines the number of parts in which it is divided.

- Unit Square Mesh - `mesh = UnitSquareMesh(Nx,Ny,"Orientation")`  
Nx defines the number of parts in which X-axis is divided.  
Ny defines the number of parts in which Y-axis is divided.  
"Orientation"defines the orientation of the diagonal it can be *left*, *right*, *left/right* and *crossed* . It is optional.

- RectangleMesh - `mesh = RectangleMesh(Point(x0, y0), Point(x1, y1), Nx, Ny, "Orientation")`  
x0,y0,x1 and y1 represents the coordinates of the two extreme points in the principal diagonal of the rectangle.
Nx, Ny and "Orinetation" holds the same meaning as above.

- Unit Cube Mesh - `UnitCubeMesh(Nx, Ny, Nz)`  
Nx,Ny and Nz holds the same meaning as above stated.  

- Box Mesh - `mesh = BoxMesh(Point(x0, y0, z0), Point(x1, y1, z1), Nx, Ny, Nz)`  
x0,y0,z0,x1,x2 and x3 defines the coordinates of the two extreme points of the principal diagonal of the box/cuboid.  
Nx,Ny and Nz holds the same meaning as above stated.






In [2]:
mesh = UnitSquareMesh(1, 1,"left")
print("Plotting a UnitSquareMesh")
plot(mesh, title="Unit square ")

Plotting a UnitSquareMesh


**The difference in "right/left" and "crossed" is that in the former one the meshes are prdocues alternatively right and left, whereas in the later one, both the diagonals are present in the each cell.**

### Create the vector function space in mesh 

To define the properties of mesh including the shape function(or element family) and degree of freedom, we use `VectorFunctionSpace`. Vector function space represents that the dof per nod is 2  
`V = VectorFunctionSpace(mesh, 'Element family', degree=x)`  
here `Element Family` - `'CG'`, `'DG'`, `'R'`, `'Q'` etc. which denotes Lagrange, Discontinuous Lagrange, Real, Quadrature respectively.  
`x` is the degree of element which can be 0, 1, 2 etc.  

One can also [refer](https://computationalmechanics.in/data-structure-of-fem-in-fenics/) this to know more about this

In [3]:
V = VectorFunctionSpace(mesh, 'Lagrange', degree=1)

### DOF to vertex map

This helps to plot the dof of all corresponding vertices. We are using the `reshape` function so as to modify the dimension.

In [4]:
v2d=vertex_to_dof_map(V)
print(v2d)
v2d = v2d.reshape((-1, mesh.geometry().dim()))

[6 7 2 3 4 5 0 1]


In [5]:
v2d

array([[6, 7],
       [2, 3],
       [4, 5],
       [0, 1]], dtype=int32)

In [6]:
num_dof = mesh.num_vertices()*V.dofmap().num_entity_dofs(0)
dof_map = Function(V,name="dof")
dof_map.vector()[:] = [int(i) for i in np.linspace(0,num_dof-1,num_dof)]

### Save the dof in a xdmf file which can be visualised in Paraview
Sometimes for complex shape with very large number of elements, it becomes extremely difficult to understand the dof for various vertex, therefore in such cases the file is exported in .xdmf format which can be visulized in Paraview.
The written command generates the `dof.xdmf` file.
* Press V to visualise the cell and point data correspondng to a specific dof. (in Paraview)

In [7]:
with XDMFFile("dof.xdmf") as xdmf:
    xdmf.write(dof_map)

Additionally one can also check number of vertices,cells and edges etc., mesh cell vertices, dof coordinates with the following commands- 
`print(dof_map.vector()[:])` - prints dof of the mesh in vector form (in general, we prefer the array form)  
`print(mesh.cells())` - prints the list of vertices for differents cells  
`V.tabulate_dof_coordinates()` - prints the dof of the the coordinates.  
`mesh.coordinates()` - prints the coordinates of the vertices.  
`mesh.num_cells()` - prints the number of cells in the mesh  
`mesh.num_edges()` - prints the number of edges in the mesh  
`mesh.num_vertices()` - prints the number of vertices in the mesh  

If one wishes to plot vertices corresponding to different edges, triangle (for 2D) , tetrahedrons (for 3D), then following commands can be used.

In [8]:
# Edges vs vertices
edge = edges(mesh)
print("Edge vs Vertex")
for i in edge:
    print(i.index(), "\t", end =" ")
    for j in vertices(i):
        print(j.index(), end =" ")
    print("\n")

Edge vs Vertex
0 	 0 1 

1 	 0 2 

2 	 1 2 

3 	 1 3 

4 	 2 3 



In [9]:
#Triangles vs vertices
print("Triangle vs Vertex")
cells = mesh.cells()
for i in range(len(cells)):
    print(i, "\t", cells[i])

Triangle vs Vertex
0 	 [0 1 2]
1 	 [1 2 3]


In [10]:
#Tetraherdons vs vertices
print("Tetrahedra vs Vertex")
cells = mesh.cells()
for i in range(len(cells)):
    print(i, "\t", cells[i])

Tetrahedra vs Vertex
0 	 [0 1 2]
1 	 [1 2 3]


In [11]:
mesh.coordinates()

array([[0., 0.],
       [1., 0.],
       [0., 1.],
       [1., 1.]])

In [12]:
print(mesh.cells())

[[0 1 2]
 [1 2 3]]


In [13]:
V.tabulate_dof_coordinates()

array([[1., 1.],
       [1., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [0., 0.]])

In [14]:
mesh.num_cells()

2

In [15]:
mesh.num_edges()

5

In [16]:
mesh.num_vertices()

4